In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import os
from zoom import ZOOM_SC

os.chdir("/slurm/home/yrd/liaolab/nieshuyang/AHBA_sc")

In [ ]:
# Prepare necessary information and load shared data
SBPs = ["UTG","SVG","MEGtheta","CMRO2","T1T2","deltaMT"]
# Optimal component numbers and optimal gene set size have been previously determined through ZOOM
best_comp_dict = {
    "UTG": 13,
    "SVG": 14,
    "MEGtheta": 12,
    "CMRO2": 15,
    "T1T2": 11,
    "deltaMT": 9
}
gene_size_dict = {
    "UTG": 150,
    "SVG": 30,
    "MEGtheta": 150,
    "CMRO2": 50,
    "T1T2": 30,
    "deltaMT": 30
}

expression = pd.read_csv("/HCPMMP/expression_HCPMMP.csv", index_col=0)
DS = pd.read_csv('/HCPMMP/DS_HCPMMP.csv', index_col=0)
DS = DS.reindex(expression.columns)
adata = sc.read_h5ad('/scRNA/adult_ctx_gss.h5ad')
adata.uns['GENE_STATS'] = DS

Now, let's perform single-cell decoding of above six SBPs with ZOOM.

In [ ]:
for SBP in SBPs:
    # Load SBP data
    SBP_parc = pd.read_csv(f"/HCPMMP/{SBP}/{SBP}_HCPMMP.csv",index_col=0)
    SBP_parc = SBP_parc[SBP_parc.index.isin(expression.index)]
    SBP_perm_parc = pd.read_csv(f"/HCPMMP/{SBP}/{SBP}_perm_HCPMMP.csv", index_col=0)
    SBP_perm_parc = SBP_perm_parc[SBP_perm_parc.index.isin(expression.index)]
    # Construct ZOOM_SC object
    zoom_ans = ZOOM_SC(
        adata, 
        expression, 
        SBP_parc, 
        SBP_perm_parc,
        best_comp = best_comp_dict[SBP],
        processed = True
    )
    zoom_ans.cv_PLSR()
    zoom_ans.get_gene_contrib()
    zoom_ans.PLS_report.to_csv(f"/HCPMMP/{SBP}/PLS_report_{SBP}.csv")

    if SBP in ["UTG","SVG","MEGtheta"]:
        res_df = pd.DataFrame(
            index=adata.obs.index,
            columns=[f"{SBP}(+)",f"{SBP}(+)_sig",f"{SBP}(-)",f"{SBP}(-)_sig"]
        )
        # Score cells and infer significant cell subpopulations
        # Positive direction of SBP
        zoom_ans.get_SBP_score(
            direction = True,
            gene_size = gene_size_dict[SBP],
            fdr_method = "group_bh",
            group = "Subcluster",
        )
        min_score = 3
        res_df[f"{SBP}(+)"] = zoom_ans.SBP_scores["norm_score"]
        zoom_ans.SBP_scores["sig_score"] = zoom_ans.SBP_scores["norm_score"].where((zoom_ans.SBP_scores["p_fdr0.1"]=='True')&(zoom_ans.SBP_scores["p_fdr0.1"]>min_score), other=np.nan)
        res_df[f"{SBP}(+)_sig"] = zoom_ans.SBP_scores["sig_score"]
        # Region enrichment analysis and DEG analysis
        zoom_ans.downstream_ans(
            min_score = min_score,
            group = "Cluster",
            region_col = "Area.v2",
            batch_col = "Dataset",
            dataset = ["Siletti2023"], # So far, only Siletti dataset provides sufficient cortical regions
            indvd_col = "Indvd"
        )
        valid_cts = list(zoom_ans.adata.uns["DEG"].keys())
        for ct in valid_cts:
            DEG_res = zoom_ans.adata.uns["DEG"][ct]
            region_res = zoom_ans.adata.uns["Region Enrichment"][ct]
            ct = ct.replace("/", "") # To avoid "/"
            DEG_res.to_csv(f"/HCPMMP/{SBP}/DEG/{SBP}(+)_{ct}.csv")
            region_res.to_csv(f"/HCPMMP/{SBP}/region/{SBP}(+)_{ct}.csv")
        # Negative direction of SBP
        zoom_ans.get_SBP_score(
            direction = False,
            gene_size = gene_size_dict[SBP],
            fdr_method = "group_bh",
            group = "Subcluster",
        )
        min_score = 3
        res_df[f"{SBP}(-)"] = zoom_ans.SBP_scores["norm_score"]
        zoom_ans.SBP_scores["sig_score"] = zoom_ans.SBP_scores["norm_score"].where((zoom_ans.SBP_scores["p_fdr0.1"]=='True')&(zoom_ans.SBP_scores["p_fdr0.1"]>min_score), other=np.nan)
        res_df[f"{SBP}(-)_sig"] = zoom_ans.SBP_scores["sig_score"]
        # Region enrichment analysis and DEG analysis
        zoom_ans.downstream_ans(
            min_score = min_score,
            group = "Cluster",
            region_col = "Area.v2",
            batch_col = "Dataset",
            dataset = ["Siletti2023"], # So far, only Siletti dataset provides sufficient cortical regions
            indvd_col = "Indvd"
        )
        valid_cts = list(zoom_ans.adata.uns["DEG"].keys())
        for ct in valid_cts:
            DEG_res = zoom_ans.adata.uns["DEG"][ct]
            region_res = zoom_ans.adata.uns["Region Enrichment"][ct]
            ct = ct.replace("/", "") # To avoid "/"
            DEG_res.to_csv(f"/HCPMMP/{SBP}/DEG/{SBP}(-)_{ct}.csv")
            region_res.to_csv(f"/HCPMMP/{SBP}/region/{SBP}(-)_{ct}.csv")
    else:
        res_df = pd.DataFrame(
            index=adata.obs.index,
            columns=[f"{SBP}(+)",f"{SBP}(+)_sig"]
        )
        # Score cells and infer significant cell subpopulations (Only for positive direction)
        zoom_ans.get_SBP_score(
            direction = True,
            gene_size = gene_size_dict[SBP],
            fdr_method = "group_bh",
            group = "Subcluster",
        )
        min_score = 3
        res_df[f"{SBP}(+)"] = zoom_ans.SBP_scores["norm_score"]
        zoom_ans.SBP_scores["sig_score"] = zoom_ans.SBP_scores["norm_score"].where((zoom_ans.SBP_scores["p_fdr0.1"]=='True')&(zoom_ans.SBP_scores["p_fdr0.1"]>min_score), other=np.nan)
        res_df[f"{SBP}(+)_sig"] = zoom_ans.SBP_scores["sig_score"]
    res_df.to_csv(f"/HCPMMP/{SBP}/{SBP}_scores.csv")

We can integrate SBP scores and calculate 95-th score for each subcluster.

In [ ]:
SBP_list1 = ["MOR","KOR","UTG","SVG","MEGtheta","CMRO2","T1T2","deltaMT"]
SBP_list2 = ["MOR(+)","KOR(+)","UTG(+)","UTG(-)","SVG(+)","SVG(-)","MEGtheta(+)","CMRO2(+)","T1T2(+)","deltaMT(+)"]
SBP_scores = []
for SBP in SBP_list1:
    SBP_score = pd.read_csv(f"/HCPMMP/{SBP}/{SBP}_scores.csv", index_col=0)
    SBP_scores.append(SBP_score)
SBP_scores = pd.concat(SBP_scores, axis=1)
SBP_scores["Subcluster"] = adata.obs["Subcluster"]
all_scores = SBP_scores[["Subcluster"]+SBP_list2]
all_scores95 = all_scores.groupby('Subcluster',observed=True)[SBP_list2].quantile(0.95)
all_scores95.to_csv("/HCPMMP/Subcluster_scores_95th.csv")

We score pathway activities for each cell through UCell. See UCell_socre.R for more information.

In [ ]:
# Pearson correlation between pathway activity scores and CMRO2 enrichment score
BP_scores = pd.read_csv("/GMT/UCell_score.csv", index_col=0)
# All cells
corrs = BP_scores.corrwith(SBP_scores["CMRO2(+)"], method="pearson")
corrs = pd.DataFrame(corrs,columns=["All"])
# For each supercluster
for supercluster in ["EX-IT","EX-NIT","INH-CGE","INH-MGE","NN"]:
    adata_sub = adata[adata.obs["Supercluster"]==supercluster]
    BP_sub = BP_scores[BP_scores.index.isin(adata_sub.obs.index)]
    SBP_scores_sub = SBP_scores[SBP_scores.index.isin(adata_sub.obs.index)]
    corr_sub = BP_sub.corrwith(SBP_scores_sub["CMRO2(+)"], method="pearson")
    corrs[supercluster] = corr_sub

O2_pathways = [
    "Cellular Respiration (GO:0045333)",
    "Oxidative Phosphorylation (GO:0006119)",
    "Tricarboxylic Acid Cycle (GO:0006099)",
    "Mitochondrial ATP Synthesis Coupled Electron Transport (GO:0042775)",
    "Energy Derivation by Oxidation of Organic Compounds (GO:0015980)",
    "Fatty Acid Beta-Oxidation (GO:0006635)"
]
corrs = corrs.loc[O2_pathways]
corrs.to_csv("/GMT/Corr_CMRO2_pathways.csv")